In [4]:
!pip install scikit-maad

  Using cached scikit_maad-1.3.12-py3-none-any.whl (142 kB)
  Using cached scikit_image-0.20.0-cp311-cp311-macosx_12_0_arm64.whl (12.6 MB)
  Using cached resampy-0.4.2-py3-none-any.whl (3.1 MB)
  Using cached numba-0.56.4.tar.gz (2.4 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/mf/mn2tnfld4370tjxj00fxczxm0000gn/T/pip-install-euht397a/numba_4ad9fb4d93e34631ad437a3fca2060d8/setup.py", line 51, in <module>
          _guard_py_ver()
        File "/private/var/folders/mf/mn2tnfld4370tjxj00fxczxm0000gn/T/pip-install-euht397a/numba_4ad9fb4d93e34631ad437a3fca2060d8/setup.py", line 48, in _guard_py_ver
          raise RuntimeError(msg.format(cur_py, min_py, max_py))
      Runtim

In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from maad.sound import (load, spectrogram,
                       remove_background, median_equalizer,
                       remove_background_morpho,
                       remove_background_along_axis, sharpness)

ModuleNotFoundError: No module named 'maad'

In [2]:
annot_df = pd.read_csv('uwb-pose-prediction/annotations.csv')
annot_df

,id,class
0,80a8dc1d-b216-469c-91b1-fc257562a883,0
1,3ab012da-927a-40d6-95f8-5a2f96dad360,0
2,eef92f84-5127-4063-a4a2-fee4035b1ac7,0
3,8d562739-6502-4a6c-8b83-e029b795f685,0
4,404c2e92-1101-4e62-83bf-a6fdb7dfcb9a,0
...,...,...
642,ea5513a7-f18a-4393-8ab4-44e574e70f2b,4
643,edf4aec3-410d-4ed3-90d2-12671bb4c625,4
644,297e1f02-66ac-4e4e-849a-da2f1103e8a0,4
645,926418bb-2e66-4c82-82b1-13a82faf73b4,4


In [ ]:
plt.bar(annot_df['class'].value_counts().index, height=annot_df['class'].value_counts().values)

In [ ]:
class_df = pd.read_csv('uwb-pose-prediction/classes.csv')
class_df

In [ ]:
annot_df[annot_df['class'] == 1].head()

In [ ]:
data_path = 'uwb-pose-prediction/train/train/'
data_id = '8dbc522d-23b7-4e25-936d-54f749f96e25.npy'
sample = np.load(f'{data_path}{data_id}')
print(sample.shape)
print("10 second x 256 frame per second = "+str(10*256))

In [ ]:
# I/Q data reader 
def IQdata(datas):
    iq_vecs = []
    for data in datas:
        data_length = len(data)
        i_vec = np.array(data[:data_length//2])
        q_vec = np.array(data[data_length//2:])
        iq_vec = i_vec + 1j*q_vec
        iq_vecs.append(iq_vec)
    return np.stack(iq_vecs)

#Range time data
def range_time(IQ_data, overlap = 1): 
    n_rd_history = 256
    frame = []
    frames = []
    overlap_count = 0
    check = 0
    for iqini in IQ_data:
        if len(frame)<n_rd_history:
            frame.append(iqini)
        else:  
            if check == 0:
                frame1 = np.array(frame)
                frames.append(np.copy(frame1))
                check = 1

            if overlap_count < overlap:
                frame.append(iqini)
                overlap_count = overlap_count+1

            if overlap_count == overlap:
                frame = frame[overlap_count::]
                frame1 = np.array(frame)
                frames.append(np.copy(frame1))
                overlap_count = 0
    return np.stack(frames)

import scipy.fftpack as fft
#Range frequency data
def range_frequency(datas):
    Range_frequency_frame = []
    for data in datas:
        jitter = 1e-10
        noise_threshold = -60
        dB = True
        #rd_history = np.hanning(n_rd_history)[:, None] * np.array(data)
        # Range-Doppler
        rd = fft.fft(data, axis=0)
        rd = fft.fftshift(rd, axes=0)
        rd = abs(rd)
        if dB:
            rd = 20 * np.log10(rd+jitter)
            rd[rd < noise_threshold] = noise_threshold
        Range_frequency_frame.append(rd)
    return np.stack(Range_frequency_frame)

def get_time_img(data_id, data_path = 'uwb-pose-prediction/train/train/'):
    output = np.load(f'{data_path}{data_id}')
    output = np.abs(range_time(output, overlap=256))
    resize = tv.transforms.Resize((256,256))
    output = resize(torch.tensor(output.reshape(1,-1,56)))
    output = np.array(torch.permute(output ,(1,2,0)))
    return output

def get_freq_img(data_id, data_path = 'uwb-pose-prediction/train/train/'):
    output = np.load(f'{data_path}{data_id}')
    output = range_time(output, overlap=256)
    output = range_frequency(output)
    resize = tv.transforms.Resize((256,256))
    output = resize(torch.tensor(output.reshape(1,-1,56)))
    output = np.array(torch.permute(output ,(1,2,0)))
    return output

In [ ]:
import torchvision as tv
import torch

plt.figure()
tmp = get_freq_img('8dbc522d-23b7-4e25-936d-54f749f96e25.npy', data_path = 'uwb-pose-prediction/train/train/')
plt.imshow(np.array(tmp))

plt.figure()
tmp = np.abs(get_time_img('8dbc522d-23b7-4e25-936d-54f749f96e25.npy', data_path = 'uwb-pose-prediction/train/train/'))
plt.imshow(np.array(tmp))

In [ ]:
# # Fall
# plt.figure()
# tmp = get_freq_img('404c2e92-1101-4e62-83bf-a6fdb7dfcb9a.npy', data_path = 'uwb-pose-prediction/train/train/')
# plt.imshow(np.array(tmp))

# plt.figure()
# tmp = np.abs(get_time_img('404c2e92-1101-4e62-83bf-a6fdb7dfcb9a.npy', data_path = 'uwb-pose-prediction/train/train/'))
# plt.imshow(np.array(tmp))

In [ ]:
# tmp = get_freq_img('b38b9347-2f91-4144-a377-b3b716d985d5.npy', data_path = 'uwb-pose-prediction/train/train/')
# for i in range(len(tmp)):
#     for j in tmp[i]:
#         if j > -45:
#             print(j)
#             continue
# tmp

# Visualization

In [ ]:
class_df['class']

## Class 0: ล้ม (เป็นลม/สะดุด)

In [ ]:
class_id = 0
data = list(annot_df[annot_df['class']==class_id].sample(10)['id'])
plt.figure(figsize=(5,10))
ax1 = plt.subplot(5,2,1)
# ax1.title.set_text(f"Time")
ax2 = plt.subplot(5,2,2)
# ax2.title.set_text(f"Freq")
ax3 = plt.subplot(5,2,3)
# ax3.title.set_text(f"Time")
ax4 = plt.subplot(5,2,4)
# ax4.title.set_text(f"Freq")
ax5 = plt.subplot(5,2,5)
# ax5.title.set_text(f"Time")
ax6 = plt.subplot(5,2,6)
# ax6.title.set_text(f"Freq")
ax7 = plt.subplot(5,2,7)
# ax7.title.set_text(f"Time")
ax8 = plt.subplot(5,2,8)
# ax8.title.set_text(f"Freq")
ax9 = plt.subplot(5,2,9)
# ax9.title.set_text(f"Time")
ax10 = plt.subplot(5,2,10)
# ax10.title.set_text(f"Freq")

im1 = ax1.imshow(get_time_img(f"{data[0]}.npy"))
im2 = ax2.imshow(get_freq_img(f"{data[0]}.npy"))
im3 = ax3.imshow(get_time_img(f"{data[1]}.npy"))
im4 = ax4.imshow(get_freq_img(f"{data[1]}.npy"))
im5 = ax5.imshow(get_time_img(f"{data[2]}.npy"))
im6 = ax6.imshow(get_freq_img(f"{data[2]}.npy"))
im7 = ax7.imshow(get_time_img(f"{data[3]}.npy"))
im8 = ax8.imshow(get_freq_img(f"{data[3]}.npy"))
im9 = ax9.imshow(get_time_img(f"{data[4]}.npy"))
im10 = ax10.imshow(get_freq_img(f"{data[4]}.npy"))

## Class 1: กระโดด

In [ ]:
class_id = 1
data = list(annot_df[annot_df['class']==class_id].sample(10)['id'])
plt.figure(figsize=(5,10))
ax1 = plt.subplot(5,2,1)
# ax1.title.set_text(f"Time")
ax2 = plt.subplot(5,2,2)
# ax2.title.set_text(f"Freq")
ax3 = plt.subplot(5,2,3)
# ax3.title.set_text(f"Time")
ax4 = plt.subplot(5,2,4)
# ax4.title.set_text(f"Freq")
ax5 = plt.subplot(5,2,5)
# ax5.title.set_text(f"Time")
ax6 = plt.subplot(5,2,6)
# ax6.title.set_text(f"Freq")
ax7 = plt.subplot(5,2,7)
# ax7.title.set_text(f"Time")
ax8 = plt.subplot(5,2,8)
# ax8.title.set_text(f"Freq")
ax9 = plt.subplot(5,2,9)
# ax9.title.set_text(f"Time")
ax10 = plt.subplot(5,2,10)
# ax10.title.set_text(f"Freq")

im1 = ax1.imshow(get_time_img(f"{data[0]}.npy"))
im2 = ax2.imshow(get_freq_img(f"{data[0]}.npy"))
im3 = ax3.imshow(get_time_img(f"{data[1]}.npy"))
im4 = ax4.imshow(get_freq_img(f"{data[1]}.npy"))
im5 = ax5.imshow(get_time_img(f"{data[2]}.npy"))
im6 = ax6.imshow(get_freq_img(f"{data[2]}.npy"))
im7 = ax7.imshow(get_time_img(f"{data[3]}.npy"))
im8 = ax8.imshow(get_freq_img(f"{data[3]}.npy"))
im9 = ax9.imshow(get_time_img(f"{data[4]}.npy"))
im10 = ax10.imshow(get_freq_img(f"{data[4]}.npy"))

## Class 2: (ตั้งใจ) ลงไปนอนบนที่นอน

In [ ]:
class_id = 2
data = list(annot_df[annot_df['class']==class_id].sample(10)['id'])
plt.figure(figsize=(5,10))
ax1 = plt.subplot(5,2,1)
# ax1.title.set_text(f"Time")
ax2 = plt.subplot(5,2,2)
# ax2.title.set_text(f"Freq")
ax3 = plt.subplot(5,2,3)
# ax3.title.set_text(f"Time")
ax4 = plt.subplot(5,2,4)
# ax4.title.set_text(f"Freq")
ax5 = plt.subplot(5,2,5)
# ax5.title.set_text(f"Time")
ax6 = plt.subplot(5,2,6)
# ax6.title.set_text(f"Freq")
ax7 = plt.subplot(5,2,7)
# ax7.title.set_text(f"Time")
ax8 = plt.subplot(5,2,8)
# ax8.title.set_text(f"Freq")
ax9 = plt.subplot(5,2,9)
# ax9.title.set_text(f"Time")
ax10 = plt.subplot(5,2,10)
# ax10.title.set_text(f"Freq")

im1 = ax1.imshow(get_time_img(f"{data[0]}.npy"))
im2 = ax2.imshow(get_freq_img(f"{data[0]}.npy"))
im3 = ax3.imshow(get_time_img(f"{data[1]}.npy"))
im4 = ax4.imshow(get_freq_img(f"{data[1]}.npy"))
im5 = ax5.imshow(get_time_img(f"{data[2]}.npy"))
im6 = ax6.imshow(get_freq_img(f"{data[2]}.npy"))
im7 = ax7.imshow(get_time_img(f"{data[3]}.npy"))
im8 = ax8.imshow(get_freq_img(f"{data[3]}.npy"))
im9 = ax9.imshow(get_time_img(f"{data[4]}.npy"))
im10 = ax10.imshow(get_freq_img(f"{data[4]}.npy"))

## Class 3: วิ่ง

In [ ]:
class_id = 3
data = list(annot_df[annot_df['class']==class_id].sample(10)['id'])
plt.figure(figsize=(5,10))
ax1 = plt.subplot(5,2,1)
# ax1.title.set_text(f"Time")
ax2 = plt.subplot(5,2,2)
# ax2.title.set_text(f"Freq")
ax3 = plt.subplot(5,2,3)
# ax3.title.set_text(f"Time")
ax4 = plt.subplot(5,2,4)
# ax4.title.set_text(f"Freq")
ax5 = plt.subplot(5,2,5)
# ax5.title.set_text(f"Time")
ax6 = plt.subplot(5,2,6)
# ax6.title.set_text(f"Freq")
ax7 = plt.subplot(5,2,7)
# ax7.title.set_text(f"Time")
ax8 = plt.subplot(5,2,8)
# ax8.title.set_text(f"Freq")
ax9 = plt.subplot(5,2,9)
# ax9.title.set_text(f"Time")
ax10 = plt.subplot(5,2,10)
# ax10.title.set_text(f"Freq")

im1 = ax1.imshow(get_time_img(f"{data[0]}.npy"))
im2 = ax2.imshow(get_freq_img(f"{data[0]}.npy"))
im3 = ax3.imshow(get_time_img(f"{data[1]}.npy"))
im4 = ax4.imshow(get_freq_img(f"{data[1]}.npy"))
im5 = ax5.imshow(get_time_img(f"{data[2]}.npy"))
im6 = ax6.imshow(get_freq_img(f"{data[2]}.npy"))
im7 = ax7.imshow(get_time_img(f"{data[3]}.npy"))
im8 = ax8.imshow(get_freq_img(f"{data[3]}.npy"))
im9 = ax9.imshow(get_time_img(f"{data[4]}.npy"))
im10 = ax10.imshow(get_freq_img(f"{data[4]}.npy"))

## Class 4: จากยืน/เดินลงไปนั่ง

In [ ]:
class_id = 4
data = list(annot_df[annot_df['class']==class_id].sample(10)['id'])
plt.figure(figsize=(5,10))
ax1 = plt.subplot(5,2,1)
# ax1.title.set_text(f"Time")
ax2 = plt.subplot(5,2,2)
# ax2.title.set_text(f"Freq")
ax3 = plt.subplot(5,2,3)
# ax3.title.set_text(f"Time")
ax4 = plt.subplot(5,2,4)
# ax4.title.set_text(f"Freq")
ax5 = plt.subplot(5,2,5)
# ax5.title.set_text(f"Time")
ax6 = plt.subplot(5,2,6)
# ax6.title.set_text(f"Freq")
ax7 = plt.subplot(5,2,7)
# ax7.title.set_text(f"Time")
ax8 = plt.subplot(5,2,8)
# ax8.title.set_text(f"Freq")
ax9 = plt.subplot(5,2,9)
# ax9.title.set_text(f"Time")
ax10 = plt.subplot(5,2,10)
# ax10.title.set_text(f"Freq")

im1 = ax1.imshow(get_time_img(f"{data[0]}.npy"))
im2 = ax2.imshow(get_freq_img(f"{data[0]}.npy"))
im3 = ax3.imshow(get_time_img(f"{data[1]}.npy"))
im4 = ax4.imshow(get_freq_img(f"{data[1]}.npy"))
im5 = ax5.imshow(get_time_img(f"{data[2]}.npy"))
im6 = ax6.imshow(get_freq_img(f"{data[2]}.npy"))
im7 = ax7.imshow(get_time_img(f"{data[3]}.npy"))
im8 = ax8.imshow(get_freq_img(f"{data[3]}.npy"))
im9 = ax9.imshow(get_time_img(f"{data[4]}.npy"))
im10 = ax10.imshow(get_freq_img(f"{data[4]}.npy"))

## Class 5: จากนั่งลุกขึ้นมายืน/เดิน

In [ ]:
class_id = 5
data = list(annot_df[annot_df['class']==class_id].sample(10)['id'])
plt.figure(figsize=(5,10))
ax1 = plt.subplot(5,2,1)
# ax1.title.set_text(f"Time")
ax2 = plt.subplot(5,2,2)
# ax2.title.set_text(f"Freq")
ax3 = plt.subplot(5,2,3)
# ax3.title.set_text(f"Time")
ax4 = plt.subplot(5,2,4)
# ax4.title.set_text(f"Freq")
ax5 = plt.subplot(5,2,5)
# ax5.title.set_text(f"Time")
ax6 = plt.subplot(5,2,6)
# ax6.title.set_text(f"Freq")
ax7 = plt.subplot(5,2,7)
# ax7.title.set_text(f"Time")
ax8 = plt.subplot(5,2,8)
# ax8.title.set_text(f"Freq")
ax9 = plt.subplot(5,2,9)
# ax9.title.set_text(f"Time")
ax10 = plt.subplot(5,2,10)
# ax10.title.set_text(f"Freq")

im1 = ax1.imshow(get_time_img(f"{data[0]}.npy"))
im2 = ax2.imshow(get_freq_img(f"{data[0]}.npy"))
im3 = ax3.imshow(get_time_img(f"{data[1]}.npy"))
im4 = ax4.imshow(get_freq_img(f"{data[1]}.npy"))
im5 = ax5.imshow(get_time_img(f"{data[2]}.npy"))
im6 = ax6.imshow(get_freq_img(f"{data[2]}.npy"))
im7 = ax7.imshow(get_time_img(f"{data[3]}.npy"))
im8 = ax8.imshow(get_freq_img(f"{data[3]}.npy"))
im9 = ax9.imshow(get_time_img(f"{data[4]}.npy"))
im10 = ax10.imshow(get_freq_img(f"{data[4]}.npy"))

## Class 6: เดิน

In [ ]:
class_id = 6
data = list(annot_df[annot_df['class']==class_id].sample(10)['id'])
plt.figure(figsize=(5,10))
ax1 = plt.subplot(5,2,1)
# ax1.title.set_text(f"Time")
ax2 = plt.subplot(5,2,2)
# ax2.title.set_text(f"Freq")
ax3 = plt.subplot(5,2,3)
# ax3.title.set_text(f"Time")
ax4 = plt.subplot(5,2,4)
# ax4.title.set_text(f"Freq")
ax5 = plt.subplot(5,2,5)
# ax5.title.set_text(f"Time")
ax6 = plt.subplot(5,2,6)
# ax6.title.set_text(f"Freq")
ax7 = plt.subplot(5,2,7)
# ax7.title.set_text(f"Time")
ax8 = plt.subplot(5,2,8)
# ax8.title.set_text(f"Freq")
ax9 = plt.subplot(5,2,9)
# ax9.title.set_text(f"Time")
ax10 = plt.subplot(5,2,10)
# ax10.title.set_text(f"Freq")

im1 = ax1.imshow(get_time_img(f"{data[0]}.npy"))
im2 = ax2.imshow(get_freq_img(f"{data[0]}.npy"))
im3 = ax3.imshow(get_time_img(f"{data[1]}.npy"))
im4 = ax4.imshow(get_freq_img(f"{data[1]}.npy"))
im5 = ax5.imshow(get_time_img(f"{data[2]}.npy"))
im6 = ax6.imshow(get_freq_img(f"{data[2]}.npy"))
im7 = ax7.imshow(get_time_img(f"{data[3]}.npy"))
im8 = ax8.imshow(get_freq_img(f"{data[3]}.npy"))
im9 = ax9.imshow(get_time_img(f"{data[4]}.npy"))
im10 = ax10.imshow(get_freq_img(f"{data[4]}.npy"))